In [40]:
import tensorflow as tf

def euclideanDistance(x, z):
    y = x-z
    distance = tf.transpose(y)*(y)
    return distance

def euclideanDistanceMultipleDimension(x,z):
    r = tf.reduce_sum(A*A, 1)
    d = tf.reduce_sum(B*B,1)
    d = tf.reshape(d, [-1, 1])
    print (d.shape)
    r = tf.reshape(r, [-1, 1])
    print (r.shape)


    D = r - 2*tf.matmul(A, tf.transpose(B)) + tf.transpose(d)
    return D

def euclideanV3(x,z):
    #get the dimension of z,x 
    z_dim = tf.shape(z) #z_dim[0] = num of rows, z_dim[1] = num of columns
    z_row = z_dim[0]
    z_col = z_dim[1]
    x_dim = tf.shape(x) 
    x_row = x_dim[0]
    x_col = x_dim[1]
    
    #duplicate each row in x by z_row times
    x_expanded = tf.tile(x,[1,z_row],name='x_expanded') #note this is a flattend array
    # get the desired shape 
    x_reshaped = tf.reshape(x_expanded,[x_row,z_row,z_col])
    
    #get the square of the difference, tensorflow broadcasts z to match the size of s
    square_difference = tf.square(x_reshaped-z)
    #reduce sum each row
    d = tf.reduce_sum(square_difference,2)
    return d
    
    
    
    
a = tf.constant([1,1,1,1], name='a')
b = tf.constant([2,5,2,2],name='b')
c = euclideanDistance(a,b)
   
sess = tf.InteractiveSession()
init = tf.global_variables_initializer()
sess.run(init)

A = tf.constant([[1, 1], [2, 2], [3, 3],[4,4]])
B= tf.constant ([[4,4],[5,5],[6,6],[4,4]])
D= euclideanV3(A,B)
print(sess.run([D]))

[array([[18, 32, 50, 18],
       [ 8, 18, 32,  8],
       [ 2,  8, 18,  2],
       [ 0,  2,  8,  0]], dtype=int32)]
